# CSCI 4022 Final Project: Movie Recommendation with K-Means

## Kate McCarthy and Téa Wright

description of project? -- also put descriptions throughout

In [72]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm
from ast import literal_eval
from sklearn.cluster import KMeans

Since, the datasets for keywords and metadata have close to 50,000 rows and the ratings has 26 million, Pandas will have better performance than Numpy, so we are loading the data in as Pandas dataframes. 

In [2]:
# load in data as csv -- should we use numpy or pandas?
keywords = pd.read_csv('data/keywords.csv')
metadata = pd.read_csv('data/movies_metadata.csv')
ratings = pd.read_csv('data/ratings_small.csv')

/Users/teaywright/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
print("Keywords shape:",keywords.shape)
print("Metadata shape:",metadata.shape)
print("Ratings shape:",ratings.shape)

Keywords shape: (46419, 2)
Metadata shape: (45466, 24)
Ratings shape: (100004, 4)


In [4]:
# test cells -- making sure data sets are comparable sizes and look right
keywords.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [5]:
metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
#make sure keywords and ratings have the same movies included -- takes size down by a lot

keywords = keywords.rename(columns={"id": "movieId", "keywords": "keywords"})
merged_df = pd.merge(keywords, ratings, on='movieId', how='inner')
ratings = merged_df.drop(columns = 'keywords')
ratings.head()

,movieId,userId,rating,timestamp
0,949,23,3.5,1148721092
1,949,102,4.0,956598942
2,949,232,2.0,955092697
3,949,242,5.0,956688825
4,949,263,3.0,1117846575


In [8]:
#Included movies -- decrease data size to 10000 ish

included_movieIds_tmp = []

for movieId in ratings['movieId']:
    included_movieIds_tmp.append(movieId)

included_movieIds = list(set(included_movieIds_tmp))

#make alphabetical
included_movieIds = sorted(included_movieIds)

print("Number of unique movies:",len(included_movieIds))
print(included_movieIds[:10])

Number of unique movies: 2830
[2, 3, 5, 6, 11, 12, 13, 14, 15, 16]


In [9]:
print("Keywords shape:",keywords.shape)
print("Metadata shape:",metadata.shape)
print("Ratings shape:",ratings.shape)

Keywords shape: (46419, 2)
Metadata shape: (45466, 24)
Ratings shape: (45010, 4)


In [10]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #crop at 5 keywords -- maybe make less
        if len(names) > 3:
            names = names[:3]
        return names
    return []

keywords['keywords'] = keywords['keywords'].apply(literal_eval)
keywords['keywords'] = keywords['keywords'].apply(get_list)

#cleaning lowercase, spaces, etc?

In [11]:
keywords.head()

,movieId,keywords
0,862,"[jealousy, toy, boy]"
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger]"
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence]"


In [12]:
#get unique keywords

master_keyword_list = []

for keywords_list in keywords['keywords']:
    master_keyword_list.append(keywords_list)
    
tmp = [item for sublist in master_keyword_list for item in sublist]
unique_keywords = list(set(tmp))

#make alphabetical
unique_keywords = sorted(unique_keywords)

print("Number of unique keywords:",len(unique_keywords))
print("First 10 keywords:", unique_keywords[:10])

Number of unique keywords: 9933
First 10 keywords: ["'comfort women'", '077', '16th century', '17th century', '18th century', '1910s', '1920s', '1930s', '1940s', '1950s']


In [13]:
#function to go from arr index to keyword string
def idx_to_keyword(idx):
    return unique_keywords[idx]
def keyword_to_idx(keyword):
    return unique_keywords.index(keyword)

In [14]:
#test idx_to_keyword and keyword_to_idx
idx_to_keyword(3)
keyword_to_idx('romance')

7545

In [15]:
#find number of users
master_user_list = []

for user in ratings['userId']:
    if user in master_user_list:
        continue
    master_user_list.append(user)


unique_users = list(set(master_user_list))
unique_users = sorted(unique_users)

print("Number of unique users:",len(unique_users))
print(unique_users[:10])

Number of unique users: 671
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [33]:
#initialize np array to hold samples x features where features are each keyword and samples are users
#each value in the arr is a user rating
#add 1 so no 0-indexing
keywords_ratings = np.zeros((len(unique_users)+1,len(unique_keywords)+1))
counts = np.zeros((len(unique_users)+1,len(unique_keywords)+1))


In [35]:
#populate array

for user_id in unique_users: #for each user
    print("User ID:",user_id)

    for movie_id in included_movieIds: #for each movie 
        #skip if the user hasn't rated the movie
        rating = ratings.loc[(ratings['movieId'] == movie_id) & (ratings['userId'] == user_id)]['rating'].tolist()
        
        if rating == []:
            continue
        else:
            rating = rating[0]
        
        
        #if a user has rated a movie more than once, only use the first instance
        #if len(rating) > 1:
         #   rating = rating[0]
            
        #print("Movie ID:", movie_id)
        #print("Movie Rating:", rating)
        
        #for keywords related to that movie
        keywords_list = keywords.loc[keywords['movieId'] == movie_id]['keywords'].tolist()[0]
        
        for keyword in keywords_list:
            #print("Keyword:",keyword)
            keyword_idx = keyword_to_idx(keyword)
            
            #update avearge score for that keyword for that user
            #print("Previous Arr Entry:",keywords_ratings[user_id][keyword_idx])
            counts[user_id][keyword_idx] += 1
            keywords_ratings[user_id][keyword_idx] = rating + (counts[user_id][keyword_idx] - 1)*keywords_ratings[user_id][keyword_idx]
            keywords_ratings[user_id][keyword_idx] /= counts[user_id][keyword_idx]   
            #print("Updated Arr Entry:", keywords_ratings[user_id][keyword_idx])


User ID: 1
User ID: 2
User ID: 3
User ID: 4
User ID: 5
User ID: 6
User ID: 7
User ID: 8
User ID: 9
User ID: 10
User ID: 11
User ID: 12
User ID: 13
User ID: 14
User ID: 15
User ID: 16
User ID: 17
User ID: 18
User ID: 19
User ID: 20
User ID: 21
User ID: 22
User ID: 23
User ID: 24
User ID: 25
User ID: 26
User ID: 27
User ID: 28
User ID: 29
User ID: 30
User ID: 31
User ID: 32
User ID: 33
User ID: 34
User ID: 35
User ID: 36
User ID: 37
User ID: 38
User ID: 39
User ID: 40
User ID: 41
User ID: 42
User ID: 43
User ID: 44
User ID: 45
User ID: 46
User ID: 47
User ID: 48
User ID: 49
User ID: 50
User ID: 51
User ID: 52
User ID: 53
User ID: 54
User ID: 55
User ID: 56
User ID: 57
User ID: 58
User ID: 59
User ID: 60
User ID: 61
User ID: 62
User ID: 63
User ID: 64
User ID: 65
User ID: 66
User ID: 67
User ID: 68
User ID: 69
User ID: 70
User ID: 71
User ID: 72
User ID: 73
User ID: 74
User ID: 75
User ID: 76
User ID: 77
User ID: 78
User ID: 79
User ID: 80
User ID: 81
User ID: 82
User ID: 83
User ID: 84
U

User ID: 640
User ID: 641
User ID: 642
User ID: 643
User ID: 644
User ID: 645
User ID: 646
User ID: 647
User ID: 648
User ID: 649
User ID: 650
User ID: 651
User ID: 652
User ID: 653
User ID: 654
User ID: 655
User ID: 656
User ID: 657
User ID: 658
User ID: 659
User ID: 660
User ID: 661
User ID: 662
User ID: 663
User ID: 664
User ID: 665
User ID: 666
User ID: 667
User ID: 668
User ID: 669
User ID: 670
User ID: 671


In [75]:
#checking
keywords_ratings = keywords_ratings.astype(int)
print(keywords_ratings.shape)
print(np.count_nonzero(keywords_ratings))

(672, 9934)
94550


In [76]:
#convert arr back to df
keywords_ratings_df = pd.DataFrame(keywords_ratings)
keywords_ratings_df.head()

,0,1,2,3,4,5,6,7,8,9,...,9924,9925,9926,9927,9928,9929,9930,9931,9932,9933
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [95]:
#cluster users
kmeans = KMeans(n_clusters=20, random_state=0)
assignments = kmeans.fit_predict(keywords_ratings, y=None, sample_weight=None)

In [96]:
print("User cluster assignments:", assignments)

User cluster assignments: [16 16 15 11 18  4 16  4 11 16 16 16 16 11 16  3 16  0  6  7  4 18  0 12
 16  6 11 16  4 16  9 11 15  4  0 16  7 16 11 15 16 18 11  4  6 16 16 15
  0  4 15 16 16 16 16  6 12 18 16 11  4 11 16 11 15 16  4 13  4 11  6 16
 11  3  4  4 16 12 12 16  6  4 15  0 11 13  7  6  0 11  6 11 13  0  0 12
 16 11 16  0  6 16  1 11 11 12 16 16 15 11 13 12  6  6 15 11 16 16  0  2
  4 13 15 16 11  0 15 16  2 16 12 16 16 16 12 16 11  4 11 16 16 16 16 11
 15 15 11 16  4 11 17 15  0 11  6  4 16  0 16  0  4 13  6 11 11 17 16 16
 13 11 16 16 15  6 16 11 11 17 11 16 16 16 13 11 15 18 16 12 13  4 16 15
 15  4 15 14  4  4 16  0 11 11 16 16 16 11 16 16 16 16 16  6 12 17 12  4
 11  4 11  0  0 16  4  6 13 15 16 16 16 16 11 16  5  6 11  0  0  4 16 18
  0 15  2 12 11  0 16 17  4 16 11 11  6 12 13 11  6 11 16 16 16 15  0  4
 16 14 15 11 12 16 11 11 16 11 16  0 16 16  4 15 16  4 11  0  6  2 16 18
 13 16  4  4 17  4  5 12 16 11 11  0 16 16 15  0  0 16  2 11  4  0 16  5
  0 12 16 16 11 15  6 16 

In [ ]:
#success metric?
#how to choose num clusters?